In [9]:
import os
from dataclasses import dataclass
from pathlib import Path
from default_predictor.constants import *
from default_predictor.utils.common import read_yaml, create_directories, get_size
#import urllib.request as request
import zipfile
from default_predictor import logger
import requests

In [7]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    token: str

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            token=config.token
        )
        
        return data_ingestion_config

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            '''filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )'''
            url = self.config.source_URL
            token = self.config.token
            headers = {
                'Authorization' : f'token {token}',
                'Accept' : 'application/vnd.github.v3.raw'
            }
            filename = url.split('/')[-1]
            filepath = self.config.root_dir
            
            response = requests.get(url=url, headers=headers)
            if response.status_code == 200:
                # Write the content to a local file
                with open(f'{filepath}/{filename}', 'wb') as file:
                    file.write(response.content)
                print("File downloaded successfully.")
            else:
                print(f"Failed to download the file. Status code: {response.status_code}")
            
            logger.info(f"{filename} downloaded.")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
                
                

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-11 13:49:50,586: INFO: common: YAML file ..\config\config.yaml loaded successfuly]
[2024-12-11 13:49:50,586: INFO: common: YAML file ..\params.yaml loaded successfuly]
[2024-12-11 13:49:50,587: INFO: common: YAML file ..\schema.yaml loaded successfuly]
[2024-12-11 13:49:50,588: INFO: common: created directory at artifacts]
[2024-12-11 13:49:50,589: INFO: common: created directory at artifacts/data_ingestion]
File downloaded successfully.
[2024-12-11 13:49:59,174: INFO: 2290217559: final_df.zip downloaded.]
